In [1]:
# take data 
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

url = 'https://voz.vn/t/event-box-cntt-2023-chia-se-kinh-nghiem-phong-van.694369/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'
}

response = urlopen(Request(url=url, headers=headers))

if response.getcode() == 200:
    soup = BeautifulSoup(response, 'html.parser')
    #DataUserId = 
    print(response.getcode())
    #DataUserName = scrapy.Field()
    #ReplyTime = scrapy.Field()
    #ReplyText = scrapy.Field()
    #QuoteBlockId = scrapy.Field()
    #QuoteBlockText = scrapy.Field()
    #print(soup.prettify())  # Hiển thị HTML đã lấy được
else:
    print(f"Lỗi khi lấy trang: {response.getcode()}")


200


In [2]:
dict_user = {}
articles = soup.find_all(
    "article", class_="message message--post js-post js-inlineModContainer"
                       
)
# Duyệt qua từng article và lấy thông tin
for article in articles:
    # Tìm thẻ con message-userDetails
    user_details = article.find("div", class_="message-userDetails")

    if user_details:
        # Tìm thẻ con chứa message-name
        message_name = user_details.find("a", class_="username")

        if message_name:
            # Lấy giá trị data-user-id và text
            user_id = message_name.get("data-user-id")  # Lấy thuộc tính data-user-id
            user_name = message_name.text  # Lấy văn bản trong thẻ
            dict_user[user_id] = user_name
print(dict_user)

{'873787': 'Fire Of Heart', '792463': 'Jarvis2802', '1772182': 'darkrose', '1841712': 'theasker01', '1786278': 'polieste', '503836': 'spam102', '1248432': 'bonamana2811', '1315878': 'dinhphong2610', '1844540': 'Mob Psycho', '1653529': 'teeeeeeeee', '1438997': 'talatroi'}


In [5]:

import re
text_csv =[]
for article in articles:
    # Tìm thẻ con message-userDetails
    user_details_content = article.find('div', class_='message-userContent lbContainer js-lbContainer')
    #print({"a":user_details.get("data-lb-caption-desc"),
    #       "b": user_details.get("data-lb-id")})               
    #block_quote = article.find('div', class_='message-userContent lbContainer js-lbContainer')
    if user_details_content:
        # Tìm thẻ con chứa message-name
        message_id = re.findall(r'\d+',user_details_content.get("data-lb-id"))[0]
        message_time = parser_time(user_details_content.get("data-lb-caption-desc"))
       
        #print(message_name)
    if user_details_content :
        blockquote = user_details_content.find('article', class_='message-body js-selectToQuote')
        
        data = {}
        blockquote1 = user_details_content.find_all('blockquote', class_='bbCodeBlock bbCodeBlock--expandable bbCodeBlock--quote js-expandWatch')
        ad_list = []
        for block in blockquote1: # one person can quote more block
                  block_id = re.findall(r'\d+',block.get("data-attributes"))
                  data_source = re.findall(r'\d+',block.get("data-source"))
                  data_text = block.get_text()
                  ad_list.append(data_source)
                  ad_list.append({"blockid":block_id[0],
                         "data_source":data_source[0],
                         "data_text":data_text})
    text_csv.append({"message_id": message_id,
               "message_time" : message_time,
               "quote" : ad_list,
              "text":blockquote.getText(),
              })
                  
       

  

In [10]:
user_details_content.get("data-lb-caption-desc")

'talatroi · Jan 6, 2023 at 9:34 AM'

In [13]:
from dateutil import parser

# Chuỗi ngày giờ cần phân tích
date_string = "Fire Of Heart · Mar 23, 2022 at 9:47 AM"

# Phân tích và chuyển đổi chuỗi ngày giờ
# Lấy phần ngày giờ từ chuỗi, ví dụ sử dụng biểu thức chính quy để trích xuất phần ngày giờ
import re

# Biểu thức chính quy để tìm phần ngày giờ
date_match = re.search(r'\b\w{3} \d{2}, \d{4} at \d{1,2}:\d{2} (AM|PM)\b', date_string)
if date_match:
    date_str = date_match.group(0)
    # Phân tích chuỗi ngày giờ thành đối tượng datetime
    date = parser.parse(date_str)
    print(date)
else:
    print("No date found in the string")


2022-03-23 09:47:00


In [16]:
from dateutil import parser
import re


def parser_time(date_string):
    # Phân tích và chuyển đổi chuỗi ngày giờ
    # Lấy phần ngày giờ từ chuỗi, ví dụ sử dụng biểu thức chính quy để trích xuất phần ngày giờ
    date_match = re.search(
        r"\b\w{3} \d{1,2}, \d{4} at \d{1,2}:\d{2} (AM|PM)\b", date_string
    )

    if date_match:
        date_str = date_match.group(0)
        # Phân tích chuỗi ngày giờ thành đối tượng datetime
        date = parser.parse(date_str)
        return date
    else:
        return "No date found in the string"

# Test lại
print(parser_time("Fire Of Heart · Mar 23, 2022 at 9:47 AM"))
print(parser_time("talatroi · Jan 6, 2023 at 9:34 AM"))


2022-03-23 09:47:00
2023-01-06 09:34:00


In [6]:
import requests

session = requests.Session()
url = 'https://voz.vn/t/co-ai-bi-sa-thai-dot-nay-ko-ver-2-viet-tat-ten-moi-cty.855701/page-185'
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
}
response = session.get(url, headers=headers)
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

In [7]:
import re

# Chuỗi đầu vào
input_string = "https---voz.vn-t-co-ai-bi-sa-thai-dot-nay-ko-ver-2-viet-tat-ten-moi-cty.855701-page-38-page=38"

# Biểu thức chính quy để trích xuất chuỗi text và số page
pattern = r"https---voz.vn-t-(.*?)-page-\d+-page=(\d+)"

# Áp dụng regex để tìm chuỗi text và số page
match = re.search(pattern, input_string)

if match:
    chuoi_text = match.group(1)  # Chuỗi text
    page_number = int(match.group(2))  # Số trang (chuyển thành số nguyên)
    
    print("Chuỗi text:", chuoi_text)
    print("Số trang:", page_number)
else:
    print("Không tìm thấy chuỗi text hoặc số trang.")


Chuỗi text: co-ai-bi-sa-thai-dot-nay-ko-ver-2-viet-tat-ten-moi-cty.855701
Số trang: 38


In [12]:
import re
pattern = r"https---voz.vn-t-(.*?)--page=(\d+)page-\d+"
url_name = "https---voz.vn-t-kiem-job-tai-faang-va-cac-big-tech-khac.594813--page=1page-1"
match = re.search(pattern, url_name)
    
if match:
        nameContent = match.group(1)
        print(nameContent)
        numberPage =  match.group(2)
        print(numberPage)

kiem-job-tai-faang-va-cac-big-tech-khac.594813
1
